In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("API_KEY")

print("done")

done


In [3]:
import requests 
import json

BASE_URL = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

input_text = "a field of blue flowers"

data = {
    "model": "deepseek/deepseek-chat:free",
    "messages": [
        {
            "role": "user",
            "content": f"Describe {input_text} in vivid detail, focusing on its appearance, movement, and sensory qualities. Use rich, descriptive language to bring the scene to life, emphasizing colors, textures, and any dynamic changes over time. Imagine the process or moment as if it were unfolding before your eyes, and convey its beauty or significance. Limit the number of words to 50"
        }
    ]
}

In [4]:
response = requests.post(BASE_URL, json=data, headers=headers)

if response.status_code != 200:
    print("Error:", response.status_code, response.text)
    sys.exit()
    

elaborated_text = response.json()["choices"][0]["message"]["content"]
print(elaborated_text)

Beneath a radiant sky, a sea of brilliant blue flowers stretches endlessly, their petals shimmering like polished sapphires in the sunlight. Gentle ripples cascade across the field as a soft breeze whispers through, stirring delicate blossoms into a fluid dance. The air hums with life, rich with the earthy scent of damp soil and sweet floral perfume, while bees dart ceaselessly between blooms. As dusk descends, the blues deepen to indigo, flecked with golden light from the fading sun. A fleeting, tranquil masterpiece.


In [5]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
import numpy as np

print("done")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

done


In [7]:
import torch
import torchvision

print(torch.__version__)
print(torch.version.cuda)
print(torchvision.__version__)

2.5.1+cu121
12.1
0.20.1+cu121


In [8]:
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

prompt = elaborated_text

model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer%2Fvocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

unet%2Fconfig.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

tokenizer%2Fspecial_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

scheduler%2Fscheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vae%2Fconfig.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/5.65G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
from PIL import Image
import numpy as np  # Import numpy for data type conversion

# Optimize memory usage
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1)
pipe.enable_vae_slicing()

#prompt = "spider-man jumping from a building to another building"

# Generate video frames first
video_frames = pipe(prompt, negative_prompt="low quality", num_inference_steps=100, num_frames=75).frames

# Convert frames from NumPy to PIL and resize
video_frames = [Image.fromarray((frame * 255).astype(np.uint8)).resize((1024, 576)) for frame in video_frames[0]]

# Export to video
video_path = export_to_video(video_frames)

Token indices sequence length is longer than the specified maximum sequence length for this model (107 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['. as dusk descends , the blues deepen to indigo , flecked with golden light from the fading sun . a fleeting , tranquil masterpiece .']


  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
import shutil
shutil.move(video_path, "/kaggle/working/output_video.mp4")  # Move to working directory

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpjaw9slly.mp4'

In [11]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00


In [ ]:
import torch
import open_clip
from torchvision import transforms
from PIL import Image
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix: Only extract model and processor
model = open_clip.create_model("ViT-B/32", pretrained="openai").to(device)
preprocess = open_clip.image_transform(model.visual.image_size, is_train=False)
tokenizer = open_clip.get_tokenizer("ViT-bigG-14")

def compute_clip_score(images, text):
    """Compute CLIP similarity score between images and text"""
    text_tokenized = tokenizer([text]).to(device)
    text_features = model.encode_text(text_tokenized).detach().cpu().numpy()

    image_scores = []
    for image in images:
        image_tensor = preprocess(image).unsqueeze(0).to(device)
        image_features = model.encode_image(image_tensor).detach().cpu().numpy()

        # Compute cosine similarity
        similarity = np.dot(image_features, text_features.T) / (np.linalg.norm(image_features) * np.linalg.norm(text_features))
        image_scores.append(similarity)

    return np.mean(image_scores)  # Average CLIP score across frames


In [17]:
clip_score = compute_clip_score(video_frames, prompt)
print(f"CLIP Score: {clip_score:.4f}")


CLIP Score: 0.3103


In [4]:
import torch
import torch.nn as nn
from transformers import CLIPTokenizer, CLIPTextModel
import imageio
import numpy as np
import os


In [5]:
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")


# **-----------------NEW CODE HERE ONWARDS-----------------**

In [6]:
# coding: utf-8

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.init as init


# see: _netD in https://github.com/pytorch/examples/blob/master/dcgan/main.py
class Discriminator_I(nn.Module):
    def __init__(self, nc=3, ndf=64, ngpu=1):
        super(Discriminator_I, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 96 x 96
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 48 x 48
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 24 x 24
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 12 x 12
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 6 x 6
            nn.Conv2d(ndf * 8, 1, 6, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)


class Discriminator_V(nn.Module):
    def __init__(self, nc=3, ndf=64, T=16, ngpu=1):
        super(Discriminator_V, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x T x 96 x 96
            nn.Conv3d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x T/2 x 48 x 48
            nn.Conv3d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x T/4 x 24 x 24
            nn.Conv3d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x T/8 x 12 x 12
            nn.Conv3d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x T/16  x 6 x 6
            Flatten(),
            nn.Linear(int((ndf*8)*(T/16)*6*6), 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)


# see: _netG in https://github.com/pytorch/examples/blob/master/dcgan/main.py
class Generator_I(nn.Module):
    def __init__(self, nc=3, ngf=64, nz=60, ngpu=1):
        super(Generator_I, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 6, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 6 x 6
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 12 x 12
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 24 x 24
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 48 x 48
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 96 x 96
        )

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output


class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0, gpu=True):
        super(GRU, self).__init__()

        output_size      = input_size
        self._gpu        = gpu
        self.hidden_size = hidden_size

        # define layers
        self.gru    = nn.GRUCell(input_size, hidden_size)
        self.drop   = nn.Dropout(p=dropout)
        self.linear = nn.Linear(hidden_size, output_size)
        self.bn     = nn.BatchNorm1d(output_size, affine=False)

    def forward(self, inputs, n_frames):
        '''
        inputs.shape()   => (batch_size, input_size)
        outputs.shape() => (seq_len, batch_size, output_size)
        '''
        outputs = []
        for i in range(n_frames):
            self.hidden = self.gru(inputs, self.hidden)
            inputs = self.linear(self.hidden)
            outputs.append(inputs)
        outputs = [ self.bn(elm) for elm in outputs ]
        outputs = torch.stack(outputs)
        return outputs

    def initWeight(self, init_forget_bias=1):
        # See details in https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/rnn.py
        for name, params in self.named_parameters():
            if 'weight' in name:
                init.xavier_uniform(params)

            # initialize forget gate bias
            elif 'gru.bias_ih_l' in name:
                b_ir, b_iz, b_in = params.chunk(3, 0)
                init.constant(b_iz, init_forget_bias)
            elif 'gru.bias_hh_l' in name:
                b_hr, b_hz, b_hn = params.chunk(3, 0)
                init.constant(b_hz, init_forget_bias)
            else:
                init.constant(params, 0)

    def initHidden(self, batch_size):
        self.hidden = Variable(torch.zeros(batch_size, self.hidden_size))
        if self._gpu == True:
            self.hidden = self.hidden.cuda()


''' utils '''

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [12]:
import torch
import os

# Initialize model instances first
image_discriminator = Discriminator_I(nc=3, ndf=64)
video_discriminator = Discriminator_V(nc=3, ndf=64, T=16)
generator = Generator_I(nc=3, ngf=64, nz=60)
gru_model = GRU(input_size=10, hidden_size=100)  # Adjust sizes based on your implementation

# Define model path
model_path = '/kaggle/input/action/pytorch/default/1'  # Adjust as needed

# Load state dictionaries
def load_model_state(model, component_name, epoch=110000):
    model_file = os.path.join(model_path, f"{component_name}_epoch-{epoch}.model")
    
    # Load the state dictionary
    state_dict = torch.load(model_file)
    
    # If the state_dict is already a state_dict, use it directly
    if isinstance(state_dict, dict):
        model.load_state_dict(state_dict)
    # If the model was saved with torch.save(model) instead of torch.save(model.state_dict())
    elif hasattr(state_dict, 'state_dict'):
        model.load_state_dict(state_dict.state_dict())
    else:
        # Handle the case where the file might contain something unexpected
        print(f"Unexpected format for {component_name}. Type: {type(state_dict)}")
    
    return model

# Load all state dictionaries into models
image_discriminator = load_model_state(image_discriminator, 'Discriminator_I')
video_discriminator = load_model_state(video_discriminator, 'Discriminator_V') 
gru_model = load_model_state(gru_model, 'GRU')
generator = load_model_state(generator, 'Generator_I')

# Now you can safely set models to evaluation mode
image_discriminator.eval()
video_discriminator.eval()
gru_model.eval()
generator.eval()


<ipython-input-12-f10ca063012f>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_file)


Generator_I(
  (main): Sequential(
    (0): ConvTranspose2d(60, 512, kernel_size=(6, 6), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [20]:
import torch
import numpy as np
import os
import imageio

# GPU setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize model instances
image_discriminator = Discriminator_I(nc=3, ndf=64).to(device)
video_discriminator = Discriminator_V(nc=3, ndf=64, T=16).to(device)
generator = Generator_I(nc=3, ngf=64, nz=60).to(device)

# For GRU, we need to define dimensions correctly
dim_z_motion = 10  # Adjust based on your implementation
gru_model = GRU(input_size=dim_z_motion, hidden_size=100).to(device)

# Load state dictionaries
model_path = '/kaggle/input/action/pytorch/default/1'

def load_model_state(model, component_name, epoch=110000):
    model_file = os.path.join(model_path, f"{component_name}_epoch-{epoch}.model")
    state_dict = torch.load(model_file, map_location=device)
    
    # If it's already a state_dict
    if isinstance(state_dict, dict):
        model.load_state_dict(state_dict)
    # If it's the full model
    elif hasattr(state_dict, 'state_dict'):
        model.load_state_dict(state_dict.state_dict())
    else:
        print(f"Unexpected format for {component_name}")
    
    return model

# Load all models
image_discriminator = load_model_state(image_discriminator, 'Discriminator_I')
video_discriminator = load_model_state(video_discriminator, 'Discriminator_V')
gru_model = load_model_state(gru_model, 'GRU')
generator = load_model_state(generator, 'Generator_I')

# Set models to evaluation mode
image_discriminator.eval()
video_discriminator.eval()
gru_model.eval()
generator.eval()

# Function to generate video

# Generate and save video
def save_video(frames, filename='generated_video.mp4', fps=8):
    imageio.mimsave(filename, frames, fps=fps)
    return filename

# Generate video
print("Generating video...")
frames = generate_video(num_frames=16)
output_file = save_video(frames, '/generated_video.mp4')
print(f"Video saved to {output_file}")


Using device: cuda


<ipython-input-20-af979ab98a17>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_file, map_location=device)


Generating video...
Video saved to /generated_video.mp4


In [21]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer

# 1. Setup text encoder
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")

# 2. Function to convert text prompt to latent vector
def prompt_to_latent(prompt, dim_z_content=50, dim_z_motion=10):
    # Encode text
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    with torch.no_grad():
        text_embeddings = text_encoder(**inputs).last_hidden_state.mean(dim=1)
    
    # Project CLIP embedding to content vector (50D)
    content_proj = torch.nn.Linear(text_embeddings.shape[1], dim_z_content)
    z_content = content_proj(text_embeddings)
    
    # Use a fixed category or derive from prompt aspects
    z_category = torch.zeros(1, 6)  # Default category vector (adjust size as needed)
    
    # Motion is typically dynamic and generated by GRU
    # Here we initialize it randomly
    z_motion = torch.randn(1, dim_z_motion)
    
    # Combine all vectors - TOTAL MUST BE 60 for your model
    z_combined = torch.cat([z_content, z_category, z_motion], dim=1)
    
    # Make sure combined vector is exactly 60 dimensions
    assert z_combined.shape[1] == 60, "Combined vector must be 60 dimensions"
    
    # Reshape for generator
    z_combined = z_combined.view(1, 60, 1, 1)
    
    return z_combined, z_motion

# 3. Generate video from prompt
def generate_video(num_frames=16, dim_z_content=50, dim_z_category=0, dim_z_motion=10):
    # Create random content vector (this would be mapped from your prompt in a text-guided system)
    z_content = torch.randn(1, dim_z_content).to(device)
    z_category = torch.zeros(1, dim_z_category).to(device)  # Can be used for category conditioning
    
    # Initialize GRU
    gru_model.initHidden(batch_size=1)
    
    # Initial motion input
    z_motion_input = torch.randn(1, dim_z_motion).to(device)
    
    # Generate frames
    frames = []
    with torch.no_grad():
        for i in range(num_frames):
            # Get motion vector from GRU
            if i == 0:
                motion = z_motion_input
            else:
                # Update motion using GRU
                motion = gru_model.gru(z_motion_input, gru_model.hidden)
                gru_model.hidden = motion
                motion = gru_model.linear(motion)
            
            # Concatenate content, category and motion vectors
            z = torch.cat([z_content, z_category, motion], dim=1)
            
            # Ensure exactly 60 dimensions for generator
            if z.size(1) > 60:
                z = z[:, :60]
            elif z.size(1) < 60:
                padding = torch.zeros(z.size(0), 60-z.size(1)).to(device)
                z = torch.cat([z, padding], dim=1)
                
            # Reshape for generator
            z = z.view(z.size(0), z.size(1), 1, 1)
            
            # Generate frame
            fake = generator(z)
            
            # Convert to image
            frame = fake.squeeze().cpu().numpy().transpose(1, 2, 0)
            frame = ((frame + 1) / 2 * 255).astype(np.uint8)  # Convert from [-1,1] to [0,255]
            frames.append(frame)
    
    return frames



In [22]:
# Update this line to match your actual path
model_path = '/kaggle/input/action/pytorch/default/1'


In [23]:
def generate_video_from_prompt(prompt, num_frames=16):
    print(f"Generating video for prompt: {prompt}")
    
    # Move text encoder to the same device as your models
    text_encoder.to(device)
    
    # Get latent vectors from the prompt
    with torch.no_grad():
        # Encode text
        inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)
        text_embeddings = text_encoder(**inputs).last_hidden_state.mean(dim=1)
        
        # Project CLIP embedding to content vector (50D)
        content_proj = torch.nn.Linear(text_embeddings.shape[1], 50).to(device)
        z_content = content_proj(text_embeddings)
        
        # Use a fixed category vector (optional conditioning)
        z_category = torch.zeros(1, 0).to(device)  # Zero dimensions for category
        
        # Initialize GRU
        gru_model.initHidden(batch_size=1)
        
        # Initial motion input
        z_motion_input = torch.randn(1, 10).to(device)
        
        # Generate frames
        frames = []
        for i in range(num_frames):
            # Get motion vector from GRU
            if i == 0:
                motion = z_motion_input
            else:
                # Update motion using GRU
                motion = gru_model.gru(z_motion_input, gru_model.hidden)
                gru_model.hidden = motion
                motion = gru_model.linear(motion)
            
            # Concatenate content, category and motion vectors
            z = torch.cat([z_content, z_category, motion], dim=1)
            
            # Ensure exactly 60 dimensions for generator
            if z.size(1) > 60:
                z = z[:, :60]
            elif z.size(1) < 60:
                padding = torch.zeros(z.size(0), 60-z.size(1)).to(device)
                z = torch.cat([z, padding], dim=1)
                
            # Reshape for generator
            z = z.view(z.size(0), z.size(1), 1, 1)
            
            # Generate frame
            fake = generator(z)
            
            # Convert to image
            frame = fake.squeeze().cpu().numpy().transpose(1, 2, 0)
            frame = ((frame + 1) / 2 * 255).astype(np.uint8)  # Convert from [-1,1] to [0,255]
            frames.append(frame)
        
        return frames


In [24]:
# Install required packages if not already installed
!pip install transformers imageio imageio-ffmpeg

# Example prompts to try
example_prompts = [
    "A person dancing",
    "A face showing surprise",
    "A person doing backflips",
    "A cat playing",
    "A hand waving"
]

# Choose a prompt or create your own
prompt = "A face showing happiness"  # Replace with your desired prompt

# Generate the video
video_frames = generate_video_from_prompt(prompt, num_frames=16)

# Save the generated video
output_path = '/kaggle/working/'  # Kaggle's writable directory
output_file = save_video(video_frames, f"{output_path}prompt_{prompt.replace(' ', '_')[:20]}.mp4")
print(f"Video saved to {output_file}")


Generating video for prompt: A face showing happiness
Video saved to /kaggle/working/prompt_A_face_showing_happi.mp4


In [25]:
def generate_variations(prompt, num_variations=3):
    for i in range(num_variations):
        # Set a different seed for each variation
        torch.manual_seed(i)
        
        # Generate video
        print(f"Generating variation {i+1} for prompt: {prompt}")
        frames = generate_video_from_prompt(prompt)
        
        # Save video
        output_file = save_video(frames, f"/kaggle/working/prompt_{prompt.replace(' ', '_')[:20]}_variation_{i+1}.mp4")
        print(f"Variation {i+1} saved to {output_file}")

# Generate multiple variations
generate_variations("A person dancing", num_variations=3)


Generating variation 1 for prompt: A person dancing
Generating video for prompt: A person dancing
Variation 1 saved to /kaggle/working/prompt_A_person_dancing_variation_1.mp4
Generating variation 2 for prompt: A person dancing
Generating video for prompt: A person dancing
Variation 2 saved to /kaggle/working/prompt_A_person_dancing_variation_2.mp4
Generating variation 3 for prompt: A person dancing
Generating video for prompt: A person dancing
Variation 3 saved to /kaggle/working/prompt_A_person_dancing_variation_3.mp4
